In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import time

: 

In [ ]:
def normalize_data(x):
    x_n = x.copy()  # Create a copy to avoid modifying the original data
    mean_values = np.mean(x_n, axis=0)
    std_values = np.std(x_n, axis=0)
    
    # Avoid division by zero
    std_values[std_values == 0] = 1
    
    x_n -= mean_values
    x_n /= std_values


    return (x_n , mean_values, std_values)

: 

In [ ]:
cancer_data = pd.read_csv("cancer_data.csv", header=None)
# Set the indices for both rows and columns
cancer_data.index = range(len(cancer_data))
cancer_data.columns = range(len(cancer_data.columns))
cancer_data


: 

In [ ]:
cancer_data_normalized, mean_values, std_values  = normalize_data(cancer_data)
cancer_data_normalized.describe()

: 

In [ ]:
expected = cancer_data_normalized.values[:, -1] # extract the y column  
cancer_data_normalized = cancer_data_normalized.values[:, :-1] # remove y column from the data

: 

In [ ]:
cancer_data_normalized_for_svd = cancer_data_normalized.copy() # save a copy from the original normalize data, will be used later
cancer_data_normalized = np.insert(cancer_data_normalized, 0, 1, axis=1)  # add intercept column

: 

* $h_{\Theta}(x^{(i)})$ is the predicted value of the $i^{th}$ observation using the hypothesis function $h_{\Theta}$

In [ ]:
def prediction (x, params): # H-teta(x)
     return np.dot(x, params)

: 

## The cost of a guess -- squared errors
$$
J(\Theta) = \frac{1}{2m} \sum_{i=1}^{n} (h_{\Theta}(x^{(i)}) - y^{(i)})^2
$$

You saw this in the classroom. In this notation:

* $m$ is the number of observations
* $n$ is the number of the attributes for each observation 
* $x^{(i)}$ is the feature vector of the $i^{th}$ observation
* $y^{(i)}$ is the actual value of the $i^{th}$ observation
* $h_{\Theta}(x^{(i)})$ is the predicted value of the $i^{th}$ observation using the hypothesis function $h_{\Theta}$
* $\Theta$ are the parameters of the model

we divide by 2m to make the math easier when we take the derivative of the cost function. It doesn't change the shape of the function. 

In [ ]:
def params_cost (X, y, params): #j(teta)
    prediction_vector = X @ params
    cost = ((y - prediction_vector) ** 2.0).sum()
    return (1.0 / (2 * len(X))) * cost


: 

## Calculating the gradient as a dependence on $\Theta$
$$
{\nabla}J(\Theta) = \frac{1}{\text(m)} \cdot (X^T \cdot (X \cdot \text(\Theta) - y))
$$

* $m$ is the number of rows in X matrix
* $x^{(i)}$ is the feature vector of the $i^{th}$ observation
* $y^{(i)}$ is the actual value of the $i^{th}$ observation
* $\Theta$ are the parameters of the model

 

In [ ]:
def gradient_calculation(X, y, params): #J-grad (teta)
    return 1/len(X) * (X.T @ (X @ params - y))
    

: 

In [ ]:
def gradient_descent(X, y, a):
    params = np.zeros(X.shape[1]) # set initial guess for the parametrs
    costs = []
    cost = params_cost(X, y, params)
    costs.append(cost)
    for _ in range(1000):
        grad = gradient_calculation(X, y, params)
        params = params - a * grad # change the parameters, in depandence of gradient and learning rate
        cost = params_cost(X, y, params)
        costs.append(cost)
        # add stopping condition, if we stable over the cost 
        if( np.abs((costs[-1] - costs[-2])) < 10e-8 ):
            break

    return (costs, params)   



: 

## Calculating the model parameters using vanilla algorithm 

In [ ]:
run_times = []
costs = [] 
a = 0.1 # set first learning rate
for _ in range(4):
    start_time = time.time()
    res = gradient_descent(cancer_data_normalized, expected, a)
    end_time = time.time()
    run_times.append(end_time - start_time)
    costs.append(res[0])
    a /= 10

x_axis = max(len(costs[0]) , len(costs[1]),len(costs[2]),len(costs[3]))

# Plotting convergence curves for different 'a' values against the number of iterations
plt.figure(figsize=(8, 6))
for i, a in enumerate([0.1, 0.01, 0.001, 0.0001]):
    plt.plot(range(len(costs[i])), costs[i],label=f'a = {a},cost = {np.round(costs[i][-1], 4)}, iterations = {len(costs[i])-1}, run-time = {np.round(run_times[i], 3)}')


plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Convergence of Gradient Descent for Different Learning Rates')
plt.legend(title='Running arguments')
plt.grid(True)
plt.show()
   

: 

## Observing the Impact of Learning Rate on Cost Convergence in Gradient Descent: Insights from Vanilla SGD Algorithm
In our analysis, we observe that the vanilla gradient descent (SGD) algorithm yields favorable cost outcomes. However, it's evident that the choice of learning rate significantly influences the convergence behavior.

For larger alpha values, we witness substantial fluctuations in the cost function, indicating significant jumps. Conversely, smaller alpha values result in more stable cost function shapes, albeit requiring a higher number of iterations to achieve satisfactory cost values

In [ ]:
def shuffle_data(X):
    # Make a copy of the input data
    x_copy = X.copy()
    shuffled_df = x_copy.sample(frac=1, random_state=42)  # frac=1 means shuffle all rows
    shuffled_df.reset_index(drop=True, inplace=True)

    return shuffled_df.to_numpy()

: 

## Calculating the model parameters using mini-batch algorithm 

In [ ]:
def batch_gradient_descent(X, y, a=0.1, batch_size=32):
    k = len(X) // batch_size
    if (len(X) % batch_size != 0):
        k += 1

    params = np.zeros(X.shape[1])
    costs = []
    costs.append(params_cost(X, y, params))

    current_index = 0  # Initialize the current index to keep track of where we are in the dataset

    for _ in range(1000):
        start_idx = current_index
        end_idx = (current_index + batch_size) % len(X)  # Compute the end index for the batch

        if end_idx < start_idx:
            indices = np.concatenate((np.arange(start_idx, len(X)), np.arange(0, end_idx)))
        else:
            indices = np.arange(start_idx, end_idx)

        x_i = X[indices]
        y_i = y[indices]

        params_0 = gradient_calculation(x_i, y_i ,params)
        params = params - a * params_0
        cost = params_cost(X, y, params)
        costs.append(cost)

        current_index = end_idx  # Update the current index for the next iteration

    return (costs, params)


: 

## Example use of mini-batch algorithm 

In [ ]:
X = shuffle_data(cancer_data)  
X, mean, std = normalize_data(X)  # Assuming normalize_data returns X, mean, std
y = X[:, -1]  # Accessing last column by index
X = X[:, :-1]   # Exclude the last column   
X = np.insert(X, 0, 1, axis=1)  # Insert column of 1s
a = 0.1
for _ in range(3):
    costs = []
    params = []
    run_times = []
    for i, b in enumerate([16, 32, 64, 128]):
        start_time = time.time()
        res = batch_gradient_descent(X, y, a, b) 
        end_time = time.time()
        costs.append(res[0])
        params.append(res[1])
        run_times.append(end_time - start_time)
        
    plt.figure(figsize=(8, 6))
    for i, b in enumerate([16, 32, 64, 128]):
        iterations = range(len(costs[i]))
        plt.plot(iterations, costs[i], label=f'b = {b} cost = { np.round(costs[i][-1], 4)} run-time = {np.round(run_times[i], 4)}')

    plt.xlabel('Iteration')
    plt.ylabel('Cost function')
    plt.title(f'Alpha = {a}')
    plt.legend(title='batch size')
    plt.grid(True)
    plt.show()

    a /= 10


: 

## Exploring Mini-Batch Gradient Descent Algorithm
Upon transitioning to the mini-batch gradient descent algorithm, notable shifts emerge, particularly in runtime metrics, while maintaining comparable cost outcomes to the vanilla approach.

The pivotal change lies in the computation of gradients, where the use of mini-batches introduces smaller multiplicative factors. Consequently, this alteration directly impacts runtime dynamics.

Furthermore, the interaction between batch size and learning rate becomes apparent. For smaller batches paired with larger learning rates, we observe less stable cost function shapes, with fluctuations being more pronounced. Conversely, larger batches with smaller learning rates yield smoother cost function trajectories.

With a bigger dataset, the impact of this algorithm becomes more pronounced, especially in the time required for tuning. It's noteworthy that while larger batch sizes may offer smoother cost function trajectories, they come at the expense of increased runtime.

In summary, leveraging the mini-batch algorithm presents an opportunity for runtime reduction. However, optimizing performance necessitates a delicate balance between batch size and learning rate, as both factors exhibit interdependence.

## Dimenstion reduction of the given data using SVD  

In [ ]:
def svd(X, dim=3):
    U, s, _ = np.linalg.svd(X, full_matrices=False)
    U_dim = U[:, :dim]
    s_dim = s[:dim]  # Create a diagonal matrix from s[:dim]
    
    X_dim = U_dim @ np.diag(s_dim)

    X_dim = np.insert(X_dim, 0, 1, axis=1)

    return X_dim


: 

## Calculating the model parameters using SGD algorithm with the reduced data

In [ ]:
cancer_data_svd = svd(cancer_data_normalized_for_svd, 3)

costs = []
run_times = []
a = 0.1
for _ in range(4):
    start_time = time.time()
    res = gradient_descent(cancer_data_svd, expected, a)
    end_time = time.time()
    run_times.append(end_time - start_time)
    costs.append(res[0])
    a /= 10

# Define the number of iterations (assuming each convergence curve has the same number of iterations)
# num_iterations = len(costs[0])
x_axis = max(len(costs[0]) , len(costs[1]),len(costs[2]),len(costs[3]))


# Plotting convergence curves for different 'a' values against the number of iterations
plt.figure(figsize=(8, 6))
for i, a in enumerate([ 0.1, 0.01, 0.001 ,0.0001]):
   plt.plot(range(len(costs[i])), costs[i],
             label=f'a = {a}, cost = {np.round(costs[i][-1], 4)}, iterations = {len(costs[i])-1}, run-time = {np.round(run_times[i], 3)}')

plt.xlabel('Iteration')
plt.ylabel('Cost function')
plt.title('Convergence of Gradient Descent for Different Learning Rates using svd')
plt.legend(title='Running arguments')
plt.grid(True)
plt.show()
   

: 

## Exploring Efficiency through Reduced Data with SVD Algorithm
Employing a reduced matrix derived from the singular value decomposition (SVD) algorithm yields remarkable reductions in runtime, despite marginal improvements in the cost function compared to the vanilla approach. This efficiency gain stems from the inherent data compression achieved through SVD.

Notably, utilizing the reduced data accelerates the convergence of the cost function, surpassing the speed observed in other algorithms. This rapid convergence significantly contributes to shorter runtimes.

Moreover, the inherent advantages of operating on smaller matrices further amplify the runtime benefits. Regardless of the algorithm used, the multiplication of reduced matrices inherently results in expedited computational processes.

In summary, the SVD algorithm offers a compelling avenue for efficiency gains, driven by data compression and accelerated convergence rates. These factors collectively lead to notable reductions in runtime, underscoring the algorithm's potential for practical applications.     

## Example use of mini-batch algorithm with the reduced data

In [ ]:
a = 0.1
for _ in range(3):
    costs = []
    params = []
    run_times = []
    for i, b in enumerate([16, 32, 64, 128]):
        start_time = time.time()
        res = batch_gradient_descent(cancer_data_svd, expected, a, b) 
        end_time = time.time()
        costs.append(res[0])
        params.append(res[1])
        run_times.append(end_time - start_time)
        
    plt.figure(figsize=(8, 6))
    for i, b in enumerate([16, 32, 64, 128]):
        iterations = range(len(costs[i]))
        plt.plot(iterations, costs[i], label=f'b = {b} cost = { np.round(costs[i][-1], 4)} run-time = {np.round(run_times[i], 4)}')

    plt.xlabel('Iteration')
    plt.ylabel('Cost function')
    plt.title(f'Alpha = {a}')
    plt.legend(title='batch size')
    plt.grid(True)
    plt.show()

    a /= 10

: 

## Deriving Model Parameters Using Direct SVD Equation Approach

In [ ]:
def params_using_svd(X,y):
    
    start_time = time.time()
    
    U, s, V = np.linalg.svd(X, full_matrices=False)
   
    theta = V.T @  (np.diag(1/s)) @ U.T @ y 
    end_time = time.time()

    cost = params_cost(X,y,theta)
    

    return ( np.round(cost,4),np.round((end_time - start_time),4) )

print(params_using_svd(cancer_data_normalized_for_svd,expected))

: 

In [ ]:
def actual_pred(prediction):
    y_std = std_values.to_numpy()[-1]
    y_mean = mean_values.to_numpy()[-1]
    return (prediction * y_std) + y_mean

cancer_data_svd = svd(cancer_data_normalized_for_svd)
params = gradient_descent(cancer_data_svd, expected, 0.01)[1]
h_t = prediction(cancer_data_svd, params)

predictions = actual_pred(h_t)
actual = actual_pred(expected)

indices = range(len(predictions))

plt.scatter(indices, predictions - actual, marker='o', color='blue')
plt.xlabel('Sample Index')
plt.ylabel('Predicted difference')
plt.title('Predictions')
plt.show()


: 

In [ ]:
X = shuffle_data(cancer_data)  
X = normalize_data(X)[0]  
y = X[:, -1]  # Accessing last column by column name
X = X[:, :-1]  # Exclude the last column   
X = np.insert(X, 0, 1, axis=1)
params = batch_gradient_descent(X, y, 0.01)[1]

h_t = prediction(cancer_data_normalized, params)

predictions = actual_pred(h_t)
actual = actual_pred(expected)

indices = range(len(predictions))

plt.scatter(indices, predictions - actual, marker='o', color='blue')
plt.xlabel('Sample Index')
plt.ylabel('Predicted difference')
plt.title('Predictions')
plt.show()

: 